In [3]:
import requests as rq
import pandas as pd
from bs4 import BeautifulSoup as bs
import re

In [4]:
import xml.etree.ElementTree as ET
from xml.etree.ElementTree import Element, dump, ElementTree

### 1. 법령 url 가져오기

In [24]:
chart= {'OC':'ICTPoolC'
        ,'target':'law'
        ,'type':'XML'
        ## 민법은 188036
        ,'MST': '188036'
        ,'JO':input()
       }

036800


* #### request 모듈을 이용하여 chart의 변수들을 전송함

In [25]:
res_chart=rq.post('http://www.law.go.kr/DRF/lawService.do?'
            ,params=chart)

* #### url 가져오기

In [26]:
print(res_chart.url)

http://www.law.go.kr/DRF/lawService.do?OC=ICTPoolC&target=law&type=XML&MST=188036&JO=036800


* #### text 가져오기
> 아래에서 볼 수 있듯이 html 타입으로 text 읽어오는건 무의미한듯..

In [27]:
print(res_chart.text)

<?xml version="1.0" encoding="UTF-8"?>
<법령 법령키="0017062016120214278">
<기본정보>
<법령ID>001706</법령ID>
<공포일자>20161202</공포일자>
<공포번호>14278</공포번호>
<언어>한글</언어>
<법종구분 법종구분코드="A0002">법률</법종구분>
<법령명_한글><![CDATA[민법]]></법령명_한글>
<법령명_한자><![CDATA[민법]]></법령명_한자>
<법령명약칭><![CDATA[]]></법령명약칭>
<제명변경여부>N</제명변경여부>
<한글법령여부>N</한글법령여부>
<편장절관>08010000</편장절관>
<소관부처 소관부처코드="1270000">법무부</소관부처>
<전화번호>02-2110-3734, 3730</전화번호>
<시행일자>20170603</시행일자>
<제개정구분>일부개정</제개정구분>
<조문별시행일자>20170603</조문별시행일자>
<별표편집여부>Y</별표편집여부>
<공포법령여부>Y</공포법령여부>
</기본정보>
<조문>
<조문단위 조문키="0368001">
<조문번호>368</조문번호>
<조문여부>조문</조문여부>
<조문제목><![CDATA[공동저당과 대가의 배당, 차순위자의 대위]]></조문제목>
<조문시행일자>20170603</조문시행일자>
<조문이동이전>000000</조문이동이전>
<조문이동이후>000000</조문이동이후>
<조문변경여부>N</조문변경여부>
			<조문내용><![CDATA[제368조(共同抵當과 代價의 配當, 次順位者의 代位)]]></조문내용>
			<항>
				<항번호><![CDATA[① ]]></항번호>
				<항내용><![CDATA[ ①동일한 채권의 담보로 수개의 부동산에 저당권을 설정한 경우에 그 부동산의 경매대가를 동시에 배당하는 때에는 각부동산의 경매대가에 비례하여 그 채권의 분담을 정한다.]]></항내용>
			</항>
			<항>
				<항번호><![CDATA[② ]]></항번호>
				<항내용><![CDATA[  ②전항의

In [12]:
res = rq.post(res_chart.url)

In [14]:
res.json()

JSONDecodeError: Expecting value: line 18 column 1 (char 34)

### 2. 크롤링으로 법령정보만 가져오기


In [82]:
soup = bs(res_chart.text,"xml")
hang = soup.find_all("항내용")

In [97]:
for i in range(len(hang)):
    print("제{}항: ".format(i+1),hang[i].text.strip()[1:])

제1항:  동일한 채권의 담보로 수개의 부동산에 저당권을 설정한 경우에 그 부동산의 경매대가를 동시에 배당하는 때에는 각부동산의 경매대가에 비례하여 그 채권의 분담을 정한다.
제2항:  전항의 저당부동산중 일부의 경매대가를 먼저 배당하는 경우에는 그 대가에서 그 채권전부의 변제를 받을 수 있다. 이 경우에 그 경매한 부동산의 차순위저당권자는 선순위저당권자가 전항의 규정에 의하여 다른 부동산의 경매대가에서 변제를 받을 수 있는 금액의 한도에서 선순위자를 대위하여 저당권을 행사할 수 있다.


### 3. 함수화


In [14]:
def Law(word):
    chart= {'OC':'ICTPoolC'
        ,'target':'law'
        ,'type':'XML'
        ## 민법은 188036
        ,'MST': '188036'
        ,'JO': word
       }
    res_chart=rq.post('http://www.law.go.kr/DRF/lawService.do?',params=chart)
    soup = bs(res_chart.text,"xml")
    hang = soup.find_all("항내용")
    print("제 {}조".format(int(word[:-2])),"\n")
    for i in range(len(hang)):
        print("제{}항: ".format(i+1),hang[i].text.strip()[1:],"\n")

In [15]:
Law("036800")

제 368조 

제1항:  동일한 채권의 담보로 수개의 부동산에 저당권을 설정한 경우에 그 부동산의 경매대가를 동시에 배당하는 때에는 각부동산의 경매대가에 비례하여 그 채권의 분담을 정한다. 

제2항:  전항의 저당부동산중 일부의 경매대가를 먼저 배당하는 경우에는 그 대가에서 그 채권전부의 변제를 받을 수 있다. 이 경우에 그 경매한 부동산의 차순위저당권자는 선순위저당권자가 전항의 규정에 의하여 다른 부동산의 경매대가에서 변제를 받을 수 있는 금액의 한도에서 선순위자를 대위하여 저당권을 행사할 수 있다. 

